In [65]:
import _pickle as pkl
import matplotlib
matplotlib.use('agg')
import matplotlib.pyplot as plt
import shapely.geometry as geo
import numpy as np
import networkx as nx
import pandas as pd
from scipy.optimize import minimize

In [66]:
# File paths
# devens_map_file = '/home/sai/TopometricRegistration/devensData/devens_map/devens_map_poly.pkl'
osm_nodes_file = '/home/sai/TopometricRegistration/devensData/devens_osm/osm_nodes.txt'
osm_edges_file = '/home/sai/TopometricRegistration/devensData/devens_osm/osm_edges.txt'
osm_nodes_gt_file = '/home/sai/TopometricRegistration/devensData/devens_osm/osm_nodes_truth.txt'
osm_edges_gt_file = '/home/sai/TopometricRegistration/devensData/devens_osm/osm_edges_truth.txt'
# cloud_file = '/home/sai/Downloads/scans_in_utm_annotated_small.pkl'
plot_dir = '/home/sai/TopometricRegistration/cache/'
full_scan_file = '/home/sai/Downloads/truth.pkl'


In [67]:
# Load in data
# devens_map = pkl.load(open(devens_map_file, 'rb'), encoding = 'bytes')
osm_nodes = np.genfromtxt(osm_nodes_file, delimiter = ',')
osm_edges = np.genfromtxt(osm_edges_file, delimiter = ',', dtype = int)
osm_nodes_gt = np.genfromtxt(osm_nodes_gt_file, delimiter = ',')
osm_edges_gt = np.genfromtxt(osm_edges_gt_file, delimiter = ',', dtype = int)

# cloud = pd.read_pickle(cloud_file)
full_scan = pd.read_pickle(full_scan_file)

In [68]:
def transform_devens_2d(x):
    osm_new = np.zeros((osm_nodes_active.shape[0], 2))
    for lol1 in range(osm_nodes_active.shape[0]):
        x_new = np.cos(x[0]) * osm_nodes_active[lol1,0] + np.sin(x[0]) * osm_nodes_active[lol1,1] + x[1]
        y_new = -np.sin(x[0]) * osm_nodes_active[lol1,0] + np.cos(x[0]) * osm_nodes_active[lol1,1] + x[2]

        osm_new[lol1,0] = x_new
        osm_new[lol1,1] = y_new
    return osm_new

In [69]:
def find_closest(x_new, y_new, scan_active):
    closest = 100000.0
    for lol2 in range(scan_active.shape[0]//100):
        dist = np.sqrt((x_new - scan_active[lol2*100,0])**2 + (y_new - scan_active[lol2*100,1])**2)
        if dist == 0:
            return dist
        if dist < closest:
            closest = dist

    return closest

In [87]:

def cost_func_2d(x):
    my_cost1 = 0
    for lol3 in range(osm_nodes_active.shape[0]):
        dx = np.random.normal(0, 0)
        dy = np.random.normal(0, 0)
        x_noise = osm_nodes_active[lol3,0] + dx
        y_noise = osm_nodes_active[lol3,1] + dy
        x_new = np.cos(x[0]) * x_noise + np.sin(x[0]) * y_noise + x[1]
        y_new = -np.sin(x[0]) * x_noise + np.cos(x[0]) * y_noise + x[2]
        my_cost1 = my_cost1 + find_closest(x_new, y_new, scan_active)
    my_cost2 = 0
    for loll3 in range(osm_nodes_active.shape[0]):
        for loll4 in range(loll3+1, osm_nodes_active.shape[0]):
            my_cost2 = my_cost2 + np.sqrt((osm_nodes_active[loll3,0] - osm_nodes_active[loll4,0])**2 + (osm_nodes_active[loll3,1] - osm_nodes_active[loll4,1])**2)
#     print(my_cost1, mycost2)
    total_cost = my_cost1 - 0.01 * my_cost2
#     print(my_cost1, my_cost2, total_cost)
    return my_cost1

In [88]:
def find_error(osm1, osm2):
    masks = np.zeros(osm2.shape[0])
    error = 0
    for lol4 in range(osm1.shape[0]):
        closest = 10000.0
        for lol5 in range(osm2.shape[0]):
            if masks[lol5] == 0 or masks[lol5] == 1:
                dist = np.sqrt((osm1[lol4,0] - osm2[lol5,0])**2 + (osm1[lol4,1] - osm2[lol5,1])**2)
                if dist == 0:
                    error = error + dist
                    masks[lol5] = 1
                    break
                if dist < closest:
                    closest = dist
                    indi = lol5
        error = error + closest
        masks[indi] = 1
    return error
                    

In [93]:
for scan_idx in range(100):
    scan = full_scan.iloc[scan_idx]['scan_odom']
    road_mask = full_scan.iloc[scan_idx]['is_road_truth']
    road_mask = np.where(road_mask)
#     xmin = min(scan['x'])
#     xmax = max(scan['x'])
#     ymin = min(scan['y'])
#     ymax = max(scan['y'])
    # print(cloud.iloc[0]['is_road_truth'])
    # print(xmin, xmax, ymin, ymax)
    # plt.plot(scan['x'], scan['y'], '.', ms=1)

    # plot_coords = lambda obj: plt.plot(obj.xy[0],obj.xy[1], 'k')
    # plot_coords(devens_map.exterior)
    # [plot_coords(x) for x in devens_map.interiors]
    # plt.plot((xmin+xmax)/2,(ymin+ymax)/2,'gx')

    # Extract OSM nodes that are within the distance threshold
    # print(full_scan.keys())
    pos_x = full_scan.iloc[scan_idx]['x']
    pos_y = full_scan.iloc[scan_idx]['y']
    
    osm_thresh = 150.
    osm_x, osm_y = [], []
    for eth in range(osm_nodes.shape[0]):
#         print(pos_x, pos_y)
#         print(osm_nodes[eth,0], osm_nodes[eth,1])
        if np.sqrt((osm_nodes[eth,0] - pos_x)**2 + (osm_nodes[eth,1] - pos_y)**2) < osm_thresh:
            osm_x.append(osm_nodes[eth,0])
            osm_y.append(osm_nodes[eth,1])
    osm_arr_x = np.asarray(osm_x)
    osm_arr_y = np.asarray(osm_y)
    osm_nodes_active = np.column_stack((osm_arr_x, osm_arr_y))
    #print(osm_nodes_active.shape)
    
    osm_gt_x, osm_gt_y = [], []
    for eth in range(osm_nodes_gt.shape[0]):
        if np.sqrt((osm_nodes_gt[eth,0] - pos_x)**2 + (osm_nodes_gt[eth,1] - pos_y)**2) < osm_thresh:
            osm_gt_x.append(osm_nodes_gt[eth,0])
            osm_gt_y.append(osm_nodes_gt[eth,1])
    osm_arr_gt_x = np.asarray(osm_gt_x)
    osm_arr_gt_y = np.asarray(osm_gt_y)
    osm_nodes_gt_active = np.column_stack((osm_arr_gt_x, osm_arr_gt_y))
    #print(osm_nodes_gt_active)
            


    # Extract LiDAR scan points that correspond to road
    scan_active = scan[road_mask]
    # print(scan_active, scan_active.shape)
    scan_active = np.asarray([[item[0] for item in scan_active], [item[1] for item in scan_active]]).T
    for btc in range(scan_active.shape[0]):
        scan_active[btc,0] = scan_active[btc,0] + pos_x
        scan_active[btc,1] = scan_active[btc,1] + pos_y
    x0 = np.array([0.1, 0.8, 1.2])
    res = minimize(cost_func_2d, x0, method='powell', options={'xtol':1e-8, 'disp':True})
    #res_bounded = minimize(cost_func_2d, x0, method='trust-constr', options={'verbose':1}, bounds=bounds)
    #print(res.x)
    osm_trans = transform_devens_2d(res.x)
    plt.scatter(osm_trans[:,0], osm_trans[:,1])
    plt.show()
    plt.savefig('/home/sai/TopometricRegistration/devensData/devens'+str(scan_idx)+'.png')
    plt.scatter(osm_nodes_gt_active[:,0], osm_nodes_gt_active[:,1])
    plt.show()
    plt.savefig('/home/sai/TopometricRegistration/devensData/road'+str(scan_idx)+'.png')
    ori_error = find_error(osm_nodes_active, osm_nodes_gt_active)
    opti_error = find_error(osm_trans, osm_nodes_gt_active)
    #print(osm_nodes_active)
    #print(osm_nodes_gt_active)
    print(ori_error, opti_error)

Optimization terminated successfully.
         Current function value: 1046.414907
         Iterations: 9
         Function evaluations: 495
75.52999072500153 582.1935301777874
Optimization terminated successfully.
         Current function value: 1060.382043
         Iterations: 11
         Function evaluations: 648
75.52999072500153 600.5659433678507
Optimization terminated successfully.
         Current function value: 909.215124
         Iterations: 10
         Function evaluations: 624
87.52910301635002 601.0259486493756
Optimization terminated successfully.
         Current function value: 1305.653084
         Iterations: 2
         Function evaluations: 141
87.52910301635002 100.85205138382096
Optimization terminated successfully.
         Current function value: 1335.413664
         Iterations: 2
         Function evaluations: 151
79.018521717122 80.61437131019747
Optimization terminated successfully.
         Current function value: 1043.047646
         Iterations: 8
         

82.30150396862628 644.8798065488596
Optimization terminated successfully.
         Current function value: 1252.221166
         Iterations: 6
         Function evaluations: 355
82.30150396862628 637.1582973729177
Optimization terminated successfully.
         Current function value: 1214.139661
         Iterations: 6
         Function evaluations: 370
107.40989507585678 671.6473526208191
Optimization terminated successfully.
         Current function value: 1195.383889
         Iterations: 7
         Function evaluations: 460
107.40989507585678 651.8315652447428
Optimization terminated successfully.
         Current function value: 1160.079888
         Iterations: 13
         Function evaluations: 747
107.40989507585678 586.2295979275942
Optimization terminated successfully.
         Current function value: 1347.896431
         Iterations: 6
         Function evaluations: 345
107.40989507585678 540.1337589727862
Optimization terminated successfully.
         Current function value: 123

KeyboardInterrupt: 